In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import pandas as pd
import numpy as np
import scipy

In [ ]:
from strategies.strategy_macross_with_trail import StrategyMACrossTrail

In [ ]:
from scripts.settings import *
#try:
#    from scripts.settings_local import *
#except:
#    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
#exo_filter = 'ES*Collar'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

#[print(exo) for exo in storage.exo_list(exo_filter)];

# Valid alpha sanity check

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': StrategyMACrossTrail,
        'exo_name': 'ES_ContFut',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,
        'direction': -1,
        'opt_params': [
                #OptParam(name, default_value, min_value, max_value, step)
                OptParamArray('Direction', [-1]),
                OptParam('SlowMAPeriod', 20, 50, 70, 10),
                OptParam('FastMAPeriod', 2, 5, 10, 5),
                OptParam('MedianPeriod', 5, 20, 30, 1)
            ],
    },    
    'swarm': {
        'members_count': 2,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.75),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()

In [ ]:
from backtester.reports.alpha_sanity_checks import AlphaSanityChecker

asc = AlphaSanityChecker(smgr)
asc.run()

# Alpha algorithm with future reference issue

In [ ]:
from backtester.analysis import *
from backtester.strategy import StrategyBase
import pandas as pd
import numpy as np

class StrategyMACrossTrailFutRef(StrategyBase):
    name = 'MACross'

    def __init__(self, strategy_context):
        # Initialize parent class
        super().__init__(strategy_context)


    @property
    def positionsize(self):
        """
        Returns volatility adjuster positions size for strategy
        :return:
        """

        # Defining EXO price
        px = self.data.exo

        # Test !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        return pd.Series(1.0, index=px.index)

        # Calculate position size adjusted to volatility of EXO
        # Dollar risk per 1 volatility unit
        risk_perunit = 100
        risk_vola_period = 100

        # Calculate volatility unit
        # In this case we use 10-period rolling median of EXO changes
        vola = abs(px.diff()).rolling(risk_vola_period).median()
        # We want to risk 100$ per 1 volatility unit
        #
        # This type of position sizing used for calibration of swarm members
        # After swarm generation and picking we will use portfolio based MM by Van Tharp
        # Tailored for portfolio size and risks of particular client
        return risk_perunit / vola

    def calculate(self, params=None, save_info=False):
        #
        #
        #  Params is a tripple like (50, 10, 15), where:
        #   50 - slow MA period
        #   10 - fast MA period
        #   15 - median period
        #
        #  On every iteration of swarming algorithm, parameter set will be different.
        #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
        #

        if params is None:
            # Return default parameters
            direction, period_slow, period_fast, period_median = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            direction, period_slow, period_fast, period_median = params

        # Defining EXO price
        px = self.data.exo

        #
        #
        # Indicator calculation
        #
        #
        slow_ma = px.rolling(period_slow).mean()
        fast_ma = px.rolling(period_fast).mean()
        # Median based trailing stop
        trailing_stop = px.rolling(period_median).median().shift(-1) # <------------------FUTURE REF!

        # Enry/exit rules
        entry_rule = CrossDown(fast_ma, slow_ma)
        exit_rule = (CrossUp(fast_ma, slow_ma)) | (CrossUp(px, trailing_stop))

        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness
        swarm_member_name = self.get_member_name(params)

        #
        # Calculation info
        #
        calc_info = None
        if save_info:
            calc_info = {'trailing_stop': trailing_stop, 'slow_ma': slow_ma, 'fast_ma': fast_ma}

        return swarm_member_name, entry_rule, exit_rule, calc_info

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': StrategyMACrossTrailFutRef,
        'exo_name': 'ES_ContFut',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,
        'direction': -1,
        'opt_params': [
                #OptParam(name, default_value, min_value, max_value, step)
                OptParamArray('Direction', [-1]),
                OptParam('SlowMAPeriod', 20, 50, 70, 10),
                OptParam('FastMAPeriod', 2, 5, 10, 5),
                OptParam('MedianPeriod', 5, 20, 30, 1)
            ],
    },    
    'swarm': {
        'members_count': 2,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.75),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()

# Sanity checks

In [ ]:
from backtester.reports.alpha_sanity_checks import AlphaSanityChecker
asc = AlphaSanityChecker(smgr)
asc.run()